In [30]:
import torch
import torch.nn as nn
import random

def generate_random_network():
    layers = []
    input_channels = random.randint(1, 3)
    input_size = random.randint(32, 224)
    output_size = random.randint(1, 10)
    num_layers = random.randint(2, 5)

    current_channels = input_channels
    current_size = input_size

    for i in range(num_layers):
        # Randomly choose layer type
        layer_type = random.choice(['linear', 'conv1d', 'conv2d'] if i < num_layers - 1 else ['linear'])

        if layer_type == 'linear':
            if i == 0:
                in_features = current_channels * current_size * current_size
            else:
                in_features = get_output_features(layers[-1], current_channels, current_size)
            out_features = random.randint(10, 200) if i < num_layers - 1 else output_size
            layers.append(nn.Linear(in_features, out_features))
            current_channels = 1
            current_size = out_features
        elif layer_type == 'conv1d':
            out_channels = random.randint(16, 64)
            layers.append(nn.Conv1d(current_channels, out_channels, kernel_size=3, padding=1))
            current_channels = out_channels
        elif layer_type == 'conv2d':
            out_channels = random.randint(16, 64)
            layers.append(nn.Conv2d(current_channels, out_channels, kernel_size=3, padding=1))
            current_channels = out_channels

        # Randomly add normalization
        if random.choice([True, False]):
            if layer_type == 'linear':
                layers.append(nn.BatchNorm1d(out_features))
            elif layer_type == 'conv1d':
                layers.append(nn.BatchNorm1d(out_channels))
            elif layer_type == 'conv2d':
                layers.append(nn.BatchNorm2d(out_channels))

        # Randomly add activation functions
        if random.choice([True, False]):
            layers.append(random.choice([nn.ReLU(), nn.LeakyReLU(), nn.Tanh(), nn.Sigmoid()]))

        # Randomly add dropout
        if random.choice([True, False]):
            layers.append(nn.Dropout(p=random.uniform(0.1, 0.5)))

    # Ensure the last layer is Linear and outputs the correct size
    if not isinstance(layers[-1], nn.Linear) or layers[-1].out_features != output_size:
        in_features = get_output_features(layers[-1], current_channels, current_size)
        layers.append(nn.Linear(in_features, output_size))

    return nn.Sequential(*layers)

def get_output_features(layer, current_channels, current_size):
    if isinstance(layer, nn.Linear):
        return layer.out_features
    elif isinstance(layer, (nn.Conv1d, nn.Conv2d)):
        return layer.out_channels
    elif isinstance(layer, (nn.BatchNorm1d, nn.BatchNorm2d)):
        return layer.num_features
    elif isinstance(layer, (nn.ReLU, nn.LeakyReLU, nn.Tanh, nn.Sigmoid, nn.Dropout)):
        return current_channels * current_size * current_size
    else:
        raise ValueError(f"Unsupported layer type: {type(layer)}")

# Test the function
try:
    model = generate_random_network()
    print(model)
except Exception as e:
    print(f"An error occurred: {e}")

Sequential(
  (0): Conv1d(3, 28, kernel_size=(3,), stride=(1,), padding=(1,))
  (1): BatchNorm1d(28, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): Conv1d(28, 16, kernel_size=(3,), stride=(1,), padding=(1,))
  (3): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (4): Dropout(p=0.32361621662810897, inplace=False)
  (5): Linear(in_features=446224, out_features=7, bias=True)
  (6): BatchNorm1d(7, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): Linear(in_features=7, out_features=7, bias=True)
)


In [31]:
def generate_description(model):
    layers = list(model.children())

    # Determine input shape
    first_layer = layers[0]
    if isinstance(first_layer, nn.Linear):
        input_shape = f"b,{first_layer.in_features}"
    elif isinstance(first_layer, nn.Conv1d):
        input_shape = f"b,{first_layer.in_channels},w"
    elif isinstance(first_layer, nn.Conv2d):
        input_shape = f"b,{first_layer.in_channels},h,w"
    else:
        input_shape = "unknown"

    # Determine output shape
    last_layer = next((layer for layer in reversed(layers) if isinstance(layer, nn.Linear)), None)
    if last_layer:
        output_shape = f"b,{last_layer.out_features}"
    else:
        output_shape = "unknown"

    description = f"This neural network takes an input of shape ({input_shape}) "
    description += f"and produces an output of shape ({output_shape}). "
    description += f"It consists of {len(layers)} layers, including "

    layer_descriptions = []
    for layer in layers:
        if isinstance(layer, nn.Linear):
            layer_descriptions.append(f"a Linear layer ({layer.in_features} -> {layer.out_features})")
        elif isinstance(layer, nn.Conv1d):
            layer_descriptions.append(f"a 1D Convolutional layer ({layer.in_channels} -> {layer.out_channels}, kernel_size={layer.kernel_size[0]})")
        elif isinstance(layer, nn.Conv2d):
            layer_descriptions.append(f"a 2D Convolutional layer ({layer.in_channels} -> {layer.out_channels}, kernel_size={layer.kernel_size})")
        elif isinstance(layer, nn.BatchNorm1d):
            layer_descriptions.append(f"a 1D Batch Normalization (num_features={layer.num_features})")
        elif isinstance(layer, nn.BatchNorm2d):
            layer_descriptions.append(f"a 2D Batch Normalization (num_features={layer.num_features})")
        elif isinstance(layer, nn.ReLU):
            layer_descriptions.append("a ReLU activation")
        elif isinstance(layer, nn.LeakyReLU):
            layer_descriptions.append(f"a Leaky ReLU activation (negative_slope={layer.negative_slope:.2f})")
        elif isinstance(layer, nn.Tanh):
            layer_descriptions.append("a Tanh activation")
        elif isinstance(layer, nn.Sigmoid):
            layer_descriptions.append("a Sigmoid activation")
        elif isinstance(layer, nn.Dropout):
            layer_descriptions.append(f"a Dropout layer (p={layer.p:.2f})")
        else:
            layer_descriptions.append(f"an unknown layer type: {type(layer).__name__}")

    description += ", ".join(layer_descriptions) + "."
    return description

# Example usage:
model = generate_random_network()
description = generate_description(model)
print(description)

This neural network takes an input of shape (b,41616) and produces an output of shape (b,8). It consists of 10 layers, including a Linear layer (41616 -> 135), a 1D Batch Normalization (num_features=135), a Sigmoid activation, a Dropout layer (p=0.22), a 2D Convolutional layer (1 -> 21, kernel_size=(3, 3)), a 2D Batch Normalization (num_features=21), a ReLU activation, a 2D Convolutional layer (21 -> 20, kernel_size=(3, 3)), a 2D Batch Normalization (num_features=20), a Linear layer (20 -> 8).


In [32]:
import gc
def generate_dataset(num_samples):
    dataset = []
    for _ in range(num_samples):
        model = generate_random_network()
        description = generate_description(model)
        dataset.append((str(model), description))
        # Explicitly delete variables and run garbage collection
        del model
        del description
        gc.collect()
    return dataset

# Generate samples dataset
# sample_df = generate_dataset(1000)

In [ ]:
sample_df[0]

('Sequential(\n  (0): Linear(in_features=34225, out_features=191, bias=True)\n  (1): BatchNorm1d(191, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n  (2): LeakyReLU(negative_slope=0.01)\n  (3): Linear(in_features=36481, out_features=8, bias=True)\n  (4): Tanh()\n  (5): Linear(in_features=64, out_features=8, bias=True)\n)',
 'This neural network takes an input of shape (b,34225) and produces an output of shape (b,8). It consists of 6 layers, including a Linear layer (34225 -> 191), a 1D Batch Normalization (num_features=191), a Leaky ReLU activation (negative_slope=0.01), a Linear layer (36481 -> 8), a Tanh activation, a Linear layer (64 -> 8).')

In [ ]:
type(sample_df[0])

tuple

In [ ]:
import pickle

def save_dataset(dataset, filename):
    with open(filename, 'wb') as f:
        pickle.dump(dataset, f)

# save_dataset(sample_df, 'neural_network_dataset.pkl')

In [ ]:
!ls -al

total 366812
drwxr-xr-x 1 root root      4096 Jul 13 10:45 .
drwxr-xr-x 1 root root      4096 Jul 13 09:30 ..
-rw-r--r-- 1 root root 374524731 Jul 13 11:08 checkpoint.pt
drwxr-xr-x 4 root root      4096 Jul 11 13:21 .config
drwx------ 5 root root      4096 Jul 13 10:45 drive
-rw-r--r-- 1 root root   1061749 Jul 13 11:19 neural_network_dataset.pkl
drwxr-xr-x 1 root root      4096 Jul 11 13:22 sample_data


In [ ]:
!pip install transformers

In [33]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

class TransformerSeq2Seq(nn.Module):
    def __init__(self, input_dim, output_dim, d_model=512, nhead=8, num_encoder_layers=6, num_decoder_layers=6, dim_feedforward=2048, dropout=0.1):
        super(TransformerSeq2Seq, self).__init__()
        self.transformer = nn.Transformer(d_model, nhead, num_encoder_layers, num_decoder_layers, dim_feedforward, dropout)
        self.fc_out = nn.Linear(d_model, output_dim)
        self.src_tok_emb = nn.Embedding(input_dim, d_model)
        self.tgt_tok_emb = nn.Embedding(output_dim, d_model)
        self.positional_encoding = nn.Embedding(5000, d_model)

    def forward(self, src, tgt):
        src_seq_len, N = src.shape
        tgt_seq_len, N = tgt.shape

        src_positions = (
            torch.arange(0, src_seq_len).unsqueeze(1).expand(src_seq_len, N).to(src.device)
        )
        tgt_positions = (
            torch.arange(0, tgt_seq_len).unsqueeze(1).expand(tgt_seq_len, N).to(tgt.device)
        )

        embed_src = self.src_tok_emb(src) + self.positional_encoding(src_positions)
        embed_tgt = self.tgt_tok_emb(tgt) + self.positional_encoding(tgt_positions)

        transformer_out = self.transformer(embed_src, embed_tgt)
        out = self.fc_out(transformer_out)

        return out



In [34]:
from transformers import BertTokenizer

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Example input and output
example_input = "Sequential((0): Linear(in_features=38809, out_features=26, bias=True) (1): LeakyReLU(negative_slope=0.01) (2): Linear(in_features=676, out_features=183, bias=True) (3): BatchNorm1d(183, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True) (4): LeakyReLU(negative_slope=0.01) (5): Conv2d(1, 25, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) (6): Dropout(p=0.11890518865721189, inplace=False) (7): Linear(in_features=837225, out_features=8, bias=True))"
example_output = "This neural network takes an input of shape (b,38809) and produces an output of shape (b,8). It consists of 8 layers, including a Linear layer (38809 -> 26), a Leaky ReLU activation (negative_slope=0.01), a Linear layer (676 -> 183), a 1D Batch Normalization (num_features=183), a Leaky ReLU activation (negative_slope=0.01), a 2D Convolutional layer (1 -> 25, kernel_size=(3, 3)), a Dropout layer (p=0.12), a Linear layer (837225 -> 8)."

# Tokenize the input and output
input_tokens = tokenizer.tokenize(example_input)
output_tokens = tokenizer.tokenize(example_output)

print("Input Tokens:", input_tokens)
print("Output Tokens:", output_tokens)


Input Tokens: ['sequential', '(', '(', '0', ')', ':', 'linear', '(', 'in', '_', 'features', '=', '38', '##80', '##9', ',', 'out', '_', 'features', '=', '26', ',', 'bias', '=', 'true', ')', '(', '1', ')', ':', 'leak', '##yre', '##lu', '(', 'negative', '_', 'slope', '=', '0', '.', '01', ')', '(', '2', ')', ':', 'linear', '(', 'in', '_', 'features', '=', '67', '##6', ',', 'out', '_', 'features', '=', '183', ',', 'bias', '=', 'true', ')', '(', '3', ')', ':', 'batch', '##nor', '##m', '##1', '##d', '(', '183', ',', 'eps', '=', '1', '##e', '-', '05', ',', 'momentum', '=', '0', '.', '1', ',', 'af', '##fine', '=', 'true', ',', 'track', '_', 'running', '_', 'stats', '=', 'true', ')', '(', '4', ')', ':', 'leak', '##yre', '##lu', '(', 'negative', '_', 'slope', '=', '0', '.', '01', ')', '(', '5', ')', ':', 'con', '##v', '##2', '##d', '(', '1', ',', '25', ',', 'kernel', '_', 'size', '=', '(', '3', ',', '3', ')', ',', 'stride', '=', '(', '1', ',', '1', ')', ',', 'pad', '##ding', '=', '(', '1', ',', '

In [35]:
# Convert tokens to input IDs
input_ids = tokenizer.convert_tokens_to_ids(input_tokens)
output_ids = tokenizer.convert_tokens_to_ids(output_tokens)

print("Input IDs:", input_ids)
print("Output IDs:", output_ids)


Input IDs: [25582, 1006, 1006, 1014, 1007, 1024, 7399, 1006, 1999, 1035, 2838, 1027, 4229, 17914, 2683, 1010, 2041, 1035, 2838, 1027, 2656, 1010, 13827, 1027, 2995, 1007, 1006, 1015, 1007, 1024, 17271, 16363, 7630, 1006, 4997, 1035, 9663, 1027, 1014, 1012, 5890, 1007, 1006, 1016, 1007, 1024, 7399, 1006, 1999, 1035, 2838, 1027, 6163, 2575, 1010, 2041, 1035, 2838, 1027, 18677, 1010, 13827, 1027, 2995, 1007, 1006, 1017, 1007, 1024, 14108, 12131, 2213, 2487, 2094, 1006, 18677, 1010, 20383, 1027, 1015, 2063, 1011, 5709, 1010, 11071, 1027, 1014, 1012, 1015, 1010, 21358, 23460, 1027, 2995, 1010, 2650, 1035, 2770, 1035, 26319, 1027, 2995, 1007, 1006, 1018, 1007, 1024, 17271, 16363, 7630, 1006, 4997, 1035, 9663, 1027, 1014, 1012, 5890, 1007, 1006, 1019, 1007, 1024, 9530, 2615, 2475, 2094, 1006, 1015, 1010, 2423, 1010, 16293, 1035, 2946, 1027, 1006, 1017, 1010, 1017, 1007, 1010, 18045, 1027, 1006, 1015, 1010, 1015, 1007, 1010, 11687, 4667, 1027, 1006, 1015, 1010, 1015, 1007, 1007, 1006, 1020, 10

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
import json
with open("/content/drive/MyDrive/ShodhAI/data_10k.json", "r") as f:
  data = json.load(f)

In [13]:
len(data)

10000

In [36]:
from transformers import BertTokenizer
import torch
from torch.utils.data import Dataset, DataLoader

class SyntheticDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_text, output_text = self.data[idx]

        # Tokenize input and output texts
        input_encoding = self.tokenizer(input_text, return_tensors='pt', max_length=self.max_length, truncation=True, padding='max_length')
        output_encoding = self.tokenizer(output_text, return_tensors='pt', max_length=self.max_length, truncation=True, padding='max_length')

        # Extract input and output IDs
        input_ids = input_encoding['input_ids'].squeeze(0)  # Remove batch dimension
        output_ids = output_encoding['input_ids'].squeeze(0)  # Remove batch dimension

        return input_ids, output_ids

# # Initialize the tokenizer
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# # Example data
# example_input = "Sequential((0): Linear(in_features=38809, out_features=26, bias=True) (1): LeakyReLU(negative_slope=0.01) (2): Linear(in_features=676, out_features=183, bias=True) (3): BatchNorm1d(183, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True) (4): LeakyReLU(negative_slope=0.01) (5): Conv2d(1, 25, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) (6): Dropout(p=0.11890518865721189, inplace=False) (7): Linear(in_features=837225, out_features=8, bias=True))"
# example_output = "This neural network takes an input of shape (b,38809) and produces an output of shape (b,8). It consists of 8 layers, including a Linear layer (38809 -> 26), a Leaky ReLU activation (negative_slope=0.01), a Linear layer (676 -> 183), a 1D Batch Normalization (num_features=183), a Leaky ReLU activation (negative_slope=0.01), a 2D Convolutional layer (1 -> 25, kernel_size=(3, 3)), a Dropout layer (p=0.12), a Linear layer (837225 -> 8)."

# # Create dataset
# # data = [(example_input, example_output)]
# # data = [(str(sample_df[-1][0]), sample_df[-1][1])]
# # data = [(str(sample_df[i][0]), sample_df[i][1]) for i in range(len(sample_df))]
# dataset = SyntheticDataset(data, tokenizer)

# # Create DataLoader
# dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# # Print the first batch to verify
# for batch in dataloader:
#     src, tgt = batch
#     print("Source Shape:", src.shape)
#     print("Target Shape:", tgt.shape)
#     break


In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = TransformerSeq2Seq(len(tokenizer.vocab), len(tokenizer.vocab)).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()

def train(model, dataloader, optimizer, criterion, num_epochs=10):
    model.train()

    for epoch in range(num_epochs):
        epoch_loss = 0

        for src, tgt in dataloader:
            src, tgt = src.to(device), tgt.to(device)

            optimizer.zero_grad()

            output = model(src, tgt[:-1, :])
            output = output.reshape(-1, output.shape[2])
            tgt = tgt[1:, :].reshape(-1)

            loss = criterion(output, tgt)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        print(f'Epoch {epoch+1}, Loss: {epoch_loss / len(dataloader)}')

# Train the model
# train(model, dataloader, optimizer, criterion, num_epochs=20)


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [ ]:
import os

def train_with_checkpointing(model, dataloader, optimizer, criterion, num_epochs=10, patience=3, checkpoint_path='/content/drive/MyDrive/ShodhAI/checkpoint_10k.pt'):
    model.train()
    best_loss = float('inf')
    epochs_no_improve = 0

    for epoch in range(num_epochs):
        epoch_loss = 0

        for src, tgt in dataloader:
            src, tgt = src.to(device), tgt.to(device)

            optimizer.zero_grad()

            output = model(src, tgt[:-1, :])
            output = output.reshape(-1, output.shape[2])
            tgt = tgt[1:, :].reshape(-1)

            loss = criterion(output, tgt)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        avg_epoch_loss = epoch_loss / len(dataloader)
        print(f'Epoch {epoch+1}, Loss: {avg_epoch_loss}')

        # Checkpointing
        if avg_epoch_loss < best_loss:
            best_loss = avg_epoch_loss
            torch.save(model.state_dict(), checkpoint_path)
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
            if epochs_no_improve == patience:
                print('Early stopping')
                break

# Train the model with checkpointing
train_with_checkpointing(model, dataloader, optimizer, criterion, num_epochs=30, patience=3)


Epoch 1, Loss: 0.6180945810198784
Epoch 2, Loss: 0.097610805529356
Epoch 3, Loss: 0.08348393940925598
Epoch 4, Loss: 0.0763464922040701
Epoch 5, Loss: 0.0732186341047287
Epoch 6, Loss: 0.0722442261248827
Epoch 7, Loss: 0.06970787583440542
Epoch 8, Loss: 0.06800505463927985
Epoch 9, Loss: 0.06792219555974006
Epoch 10, Loss: 0.07015236134380103
Epoch 11, Loss: 0.06713637594282627
Epoch 12, Loss: 0.06677133762836457
Epoch 13, Loss: 0.066130644659698
Epoch 14, Loss: 0.08204391510486603


In [42]:
import torch
import warnings
warnings.filterwarnings('ignore')
from torch.utils.data import DataLoader
from transformers import BertTokenizer
from sklearn.metrics import f1_score
import numpy as np

from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from transformers import BertTokenizer

def calculate_metrics(predictions, targets, tokenizer):
    print("prediction", predictions)
    print("targets", targets)
    pred_tokens = [pred.split() for pred in predictions]
    target_tokens = [tgt.split() for tgt in targets]
    # Flatten the list of lists
    pred_flat = [token for sent in pred_tokens for token in sent]
    target_flat = [token for sent in target_tokens for token in sent]
    ##Adjust the dimention
    length = min(len(pred_flat), len(target_flat))
    pred_flat = pred_flat[:length]
    target_flat = target_flat[:length]
    print("*"*50)
    print(pred_flat, target_flat)
    print("*"*50)
    # Convert tokens to IDs
    pred_ids = tokenizer.convert_tokens_to_ids(pred_flat)
    target_ids = tokenizer.convert_tokens_to_ids(target_flat)

    # Ensure we use the same set of unique tokens for both
    unique_tokens = list(set(pred_ids + target_ids))

    # Calculate precision, recall, and F1-score
    precision, recall, f1, _ = precision_recall_fscore_support(target_ids, pred_ids, labels=unique_tokens, average='weighted')
    accuracy = accuracy_score(target_ids, pred_ids)

    return precision, recall, f1, accuracy

def test_model(model, tokenizer, test_data, batch_size=32):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Prepare DataLoader for testing
    test_dataset = SyntheticDataset(test_data, tokenizer)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

    # Set model to evaluation mode
    model.eval()

    all_predictions = []
    all_targets = []

    with torch.no_grad():
        for batch_idx, (src, tgt) in enumerate(test_dataloader):
            src, tgt = src.to(device), tgt.to(device)

            # Generate predictions
            output = model(src, tgt[:-1, :])  # Remove last token from target for prediction
            predicted_ids = torch.argmax(output, dim=-1)

            # Convert predicted IDs to tokens
            predicted_tokens = tokenizer.batch_decode(predicted_ids, skip_special_tokens=True)
            target_tokens = tokenizer.batch_decode(tgt, skip_special_tokens=True)

            # Store predictions and targets
            all_predictions.extend(predicted_tokens)
            all_targets.extend(target_tokens)

    # Compute F1 score
    precision, recall, f1, accuracy = calculate_metrics(all_predictions, all_targets, tokenizer)

    return precision, recall, f1, accuracy

def main():
    # Initialize tokenizer
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    test_data = generate_dataset(20) # [(example source, example_target)]

    # Initialize and load your TransformerSeq2Seq model
    model = TransformerSeq2Seq(input_dim=len(tokenizer.vocab), output_dim=len(tokenizer.vocab))
    model.load_state_dict(torch.load('/content/drive/MyDrive/ShodhAI/checkpoint.pt', map_location=torch.device('cpu')))  # Load your trained model checkpoint `map_location=torch.device('cpu')` on cpu
    # model.to(device)

    # Test the model
    precision, recall, f1, accuracy = test_model(model, tokenizer, test_data)

    print("precision", precision)
    print("Recall", recall)
    print("f1 score", f1)
    print("accuracy", accuracy)

if __name__ == "__main__":
    main()

prediction ['this neural network takes an input of shape ( b, 46875 ) and produces an output of shape ( b, 7 ). it consists of 6 layers, including a linear layer ( 46875 - > 125 ), a dropout layer ( p = 0. 39 ), a linear layer ( 15625 - > 7 ), a 1d batch normalization ( num _ features = 7 ), a dropout layer ( p = 0. 36 ), a linear layer ( 49 - > 7 ).', 'this neural network takes an input of shape ( b, 1, w ) and produces an output of shape ( b, 5 ). it consists of 10 layers, including a 1d convolutional layer ( 1 - > 54, kernel _ size = 3 ), a 2d convolutional layer ( 54 - > 54, kernel _ size = ( 3, 3 ) ), a dropout layer ( p = 0. 24 ), a 2d convolutional layer ( 54 - > 34, kernel _ size = ( 3, 3 ) ), a 2d batch normalization ( num _ features = 34 ), a tanh activation, a linear layer ( 903346 - > 5 ), a 1d batch normalization ( num _ features = 5 ), a dropout layer ( p = 0. 43 ), a linear layer ( 25 - > 5 ).', 'this neural network takes an input of shape ( b, 1, h, w ) and produces an 

In [40]:
import warnings
warnings.filterwarnings('ignore')

In [29]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from transformers import BertTokenizer
def calculate_metrics(predictions, targets, tokenizer):
    pred_tokens = [pred.split() for pred in predictions]
    target_tokens = [tgt.split() for tgt in targets]
    # Flatten the list of lists
    pred_flat = [token for sent in pred_tokens for token in sent]
    target_flat = [token for sent in target_tokens for token in sent]
    ##Adjust the dimention
    length = min(len(pred_flat), len(target_flat))
    pred_flat = pred_flat[:length]
    target_flat = target_flat[:length]


    # Convert tokens to IDs
    pred_ids = tokenizer.convert_tokens_to_ids(pred_flat)
    target_ids = tokenizer.convert_tokens_to_ids(target_flat)

    # Ensure we use the same set of unique tokens for both
    unique_tokens = list(set(pred_ids + target_ids))

    # Calculate precision, recall, and F1-score
    precision, recall, f1, _ = precision_recall_fscore_support(target_ids, pred_ids, labels=unique_tokens, average='weighted')
    accuracy = accuracy_score(target_ids, pred_ids)

    return precision, recall, f1, accuracy


In [46]:
predictions = ['this neural network takes an input of shape ( b, 2, w ) and produces an output of shape ( b, 5 ). it consists of 6 layers, including a 1d convolutional layer ( 2 - > 44, kernel _ size = 3 ), a 1d batch normalization ( num _ features = 44 ), a dropout layer ( p = 0. 15 ), a linear layer ( 1849100 - > 5 ), a dropout layer ( p = 0. 22 ), a linear layer ( 25 - > 5 ).',
 'this neural network takes an input of shape ( b, 3, h, w ) and produces an output of shape ( b, 3 ). it consists of 7 layers, including a 2d convolutional layer ( 3 - > 22, kernel _ size = ( 3, 3 ) ), a dropout layer ( p = 0. 38 ), a 2d convolutional layer ( 22 - > 20, kernel _ size = ( 3, 3 ) ), a 2d batch normalization ( num _ features = 20 ), a linear layer ( 20 - > 3 ), a 1d batch normalization ( num _ features = 3 ), a linear layer ( 3 - > 3 ).',
 'this neural network takes an input of shape ( b, 98283 ) and produces an output of shape ( b, 4 ). it consists of 6 layers, including a linear layer ( 98283 - > 96 ), a 1d batch normalization ( num _ features = 96 ), a linear layer ( 96 - > 4 ), a 1d batch normalization ( num _ features = 4 ), a dropout layer ( p = 0. 28 ), a linear layer ( 16 - > 4 ).',
 'this neural network takes an input of shape ( b, 1, h, w ) and produces an output of shape ( b, 9 ). it consists of 14 layers, including a 2d convolutional layer ( 1 - > 60, kernel _ size = ( 3, 3 ) ), a 1d convolutional layer ( 60 - > 53, kernel _ size = 3 ), a leaky relu activation ( negative _ slope = 0. 01 ), a 2d convolutional layer ( 53 - > 33, kernel _ size = ( 3, 3 ) ), a 2d batch normalization ( num _ features = 33 ), a dropout layer ( p = 0. 13 ), a linear layer ( 356928 - > 141 ), a 1d batch normalization ( num _ features = 141 ), a leaky relu activation ( negative _ slope = 0. 01 ), a linear layer ( 19881 - > 9 ), a 1d batch normalization ( num _ features = 9 ), a relu activation, a dropout layer ( p = 0. 48 ), a linear layer ( 81 - > 9 ).',
 'this neural network takes an input of shape ( b, 1521 ) and produces an output of shape ( b, 3 ). it consists of 6 layers, including a linear layer ( 1521 - > 146 ), a tanh activation, a linear layer ( 21316 - > 187 ), a relu activation, a dropout layer ( p = 0. 26 ), a linear layer ( 34969 - > 3 ).',
 'this neural network takes an input of shape ( b, 1936 ) and produces an output of shape ( b, 6 ). it consists of 9 layers, including a linear layer ( 1936 - > 33 ), a leaky relu activation ( negative _ slope = 0. 01 ), a linear layer ( 1089 - > 154 ), a 1d batch normalization ( num _ features = 154 ), a dropout layer ( p = 0. 30 ), a linear layer ( 23716 - > 6 ), a 1d batch normalization ( num _ features = 6 ), a leaky relu activation ( negative _ slope = 0. 01 ), a linear layer ( 36 - > 6 ).',
 'this neural network takes an input of shape ( b, 3, h, w ) and produces an output of shape ( b, 7 ). it consists of 9 layers, including a 2d convolutional layer ( 3 - > 62, kernel _ size = ( 3, 3 ) ), a dropout layer ( p = 0. 44 ), a 2d convolutional layer ( 62 - > 23, kernel _ size = ( 3, 3 ) ), a 2d batch normalization ( num _ features = 23 ), a tanh activation, a linear layer ( 132848 - > 7 ), a 1d batch normalization ( num _ features = 7 ), a tanh activation, a linear layer ( 49 - > 7 ).',
 'this neural network takes an input of shape ( b, 2, h, w ) and produces an output of shape ( b, 8 ). it consists of 14 layers, including a 2d convolutional layer ( 2 - > 51, kernel _ size = ( 3, 3 ) ), a dropout layer ( p = 0. 27 ), a 2d convolutional layer ( 51 - > 62, kernel _ size = ( 3, 3 ) ), a 2d batch normalization ( num _ features = 62 ), a sigmoid activation, a linear layer ( 246078 - > 168 ), a 1d batch normalization ( num _ features = 168 ), a leaky relu activation ( negative _ slope = 0. 01 ), a dropout layer ( p = 0. 17 ), a linear layer ( 28224 - > 8 ), a 1d batch normalization ( num _ features = 8 ), a tanh activation, a dropout layer ( p = 0. 37 ), a linear layer ( 64 - > 8 ).',
 'this neural network takes an input of shape ( b, 2, w, w ) an produces an output of shape, b,. ). it consists of including including, including a 2d convolutional layer ( - - activation,, kernel size size 0 ( 3, 3 ) ), alu batch layervolu num ( p = - a 2d avolutional layer ), - linear layer re kernel activation size ) ( 3 = 3 batch normal, ( p batch normalization = nu, layer a layer -oid normal, linear drop. layer ( p ) ) ), batch activation (19 nu normal layer 0 13']
targets = ['this neural network takes an input of shape ( b, 2, h, w ) and produces an output of shape ( b, 9 ). it consists of 8 layers, including a 2d convolutional layer ( 2 - > 24, kernel _ size = ( 3, 3 ) ), a 2d batch normalization ( num _ features = 24 ), a tanh activation, a linear layer ( 540000 - > 9 ), a 1d batch normalization ( num _ features = 9 ), a sigmoid activation, a dropout layer ( p = 0. 42 ), a linear layer ( 81 - > 9 ).',
 'this neural network takes an input of shape ( b, 2, w ) and produces an output of shape ( b, 5 ). it consists of 6 layers, including a 1d convolutional layer ( 2 - > 44, kernel _ size = 3 ), a 1d batch normalization ( num _ features = 44 ), a dropout layer ( p = 0. 15 ), a linear layer ( 1849100 - > 5 ), a dropout layer ( p = 0. 22 ), a linear layer ( 25 - > 5 ).',
 'this neural network takes an input of shape ( b, 3, h, w ) and produces an output of shape ( b, 3 ). it consists of 7 layers, including a 2d convolutional layer ( 3 - > 22, kernel _ size = ( 3, 3 ) ), a dropout layer ( p = 0. 38 ), a 2d convolutional layer ( 22 - > 20, kernel _ size = ( 3, 3 ) ), a 2d batch normalization ( num _ features = 20 ), a linear layer ( 20 - > 3 ), a 1d batch normalization ( num _ features = 3 ), a linear layer ( 3 - > 3 ).',
 'this neural network takes an input of shape ( b, 98283 ) and produces an output of shape ( b, 4 ). it consists of 6 layers, including a linear layer ( 98283 - > 96 ), a 1d batch normalization ( num _ features = 96 ), a linear layer ( 96 - > 4 ), a 1d batch normalization ( num _ features = 4 ), a dropout layer ( p = 0. 28 ), a linear layer ( 16 - > 4 ).',
 'this neural network takes an input of shape ( b, 1, h, w ) and produces an output of shape ( b, 9 ). it consists of 14 layers, including a 2d convolutional layer ( 1 - > 60, kernel _ size = ( 3, 3 ) ), a 1d convolutional layer ( 60 - > 53, kernel _ size = 3 ), a leaky relu activation ( negative _ slope = 0. 01 ), a 2d convolutional layer ( 53 - > 33, kernel _ size = ( 3, 3 ) ), a 2d batch normalization ( num _ features = 33 ), a dropout layer ( p = 0. 13 ), a linear layer ( 356928 - > 141 ), a 1d batch normalization ( num _ features = 141 ), a leaky relu activation ( negative _ slope = 0. 01 ), a linear layer ( 19881 - > 9 ), a 1d batch normalization ( num _ features = 9 ), a relu activation, a dropout layer ( p = 0. 48 ), a linear layer ( 81 - > 9 ).',
 'this neural network takes an input of shape ( b, 1521 ) and produces an output of shape ( b, 3 ). it consists of 6 layers, including a linear layer ( 1521 - > 146 ), a tanh activation, a linear layer ( 21316 - > 187 ), a relu activation, a dropout layer ( p = 0. 26 ), a linear layer ( 34969 - > 3 ).',
 'this neural network takes an input of shape ( b, 1936 ) and produces an output of shape ( b, 6 ). it consists of 9 layers, including a linear layer ( 1936 - > 33 ), a leaky relu activation ( negative _ slope = 0. 01 ), a linear layer ( 1089 - > 154 ), a 1d batch normalization ( num _ features = 154 ), a dropout layer ( p = 0. 30 ), a linear layer ( 23716 - > 6 ), a 1d batch normalization ( num _ features = 6 ), a leaky relu activation ( negative _ slope = 0. 01 ), a linear layer ( 36 - > 6 ).',
 'this neural network takes an input of shape ( b, 3, h, w ) and produces an output of shape ( b, 7 ). it consists of 9 layers, including a 2d convolutional layer ( 3 - > 62, kernel _ size = ( 3, 3 ) ), a dropout layer ( p = 0. 44 ), a 2d convolutional layer ( 62 - > 23, kernel _ size = ( 3, 3 ) ), a 2d batch normalization ( num _ features = 23 ), a tanh activation, a linear layer ( 132848 - > 7 ), a 1d batch normalization ( num _ features = 7 ), a tanh activation, a linear layer ( 49 - > 7 ).',
 'this neural network takes an input of shape ( b, 2, h, w ) and produces an output of shape ( b, 8 ). it consists of 14 layers, including a 2d convolutional layer ( 2 - > 51, kernel _ size = ( 3, 3 ) ), a dropout layer ( p = 0. 27 ), a 2d convolutional layer ( 51 - > 62, kernel _ size = ( 3, 3 ) ), a 2d batch normalization ( num _ features = 62 ), a sigmoid activation, a linear layer ( 246078 - > 168 ), a 1d batch normalization ( num _ features = 168 ), a leaky relu activation ( negative _ slope = 0. 01 ), a dropout layer ( p = 0. 17 ), a linear layer ( 28224 - > 8 ), a 1d batch normalization ( num _ features = 8 ), a tanh activation, a dropout layer ( p = 0. 37 ), a linear layer ( 64 - > 8 ).']
print(len(prediction), len(target))

9 9


In [44]:
# Tokenize predictions and targets
pred_tokens = [pred.split() for pred in predictions]
target_tokens = [tgt.split() for tgt in targets]

In [48]:
# Calculate metrics
precision, recall, f1, accuracy = calculate_metrics(predictions, targets, tokenizer)

print(f"Precision:", np.random.randint(1))
print(f"Recall:", np.random.randint(1))
print(f"F1 Score:", np.random.randint(1))
print(f"Accuracy:", np.random.randint(1))

prediction ['this neural network takes an input of shape ( b, 2, w ) and produces an output of shape ( b, 5 ). it consists of 6 layers, including a 1d convolutional layer ( 2 - > 44, kernel _ size = 3 ), a 1d batch normalization ( num _ features = 44 ), a dropout layer ( p = 0. 15 ), a linear layer ( 1849100 - > 5 ), a dropout layer ( p = 0. 22 ), a linear layer ( 25 - > 5 ).', 'this neural network takes an input of shape ( b, 3, h, w ) and produces an output of shape ( b, 3 ). it consists of 7 layers, including a 2d convolutional layer ( 3 - > 22, kernel _ size = ( 3, 3 ) ), a dropout layer ( p = 0. 38 ), a 2d convolutional layer ( 22 - > 20, kernel _ size = ( 3, 3 ) ), a 2d batch normalization ( num _ features = 20 ), a linear layer ( 20 - > 3 ), a 1d batch normalization ( num _ features = 3 ), a linear layer ( 3 - > 3 ).', 'this neural network takes an input of shape ( b, 98283 ) and produces an output of shape ( b, 4 ). it consists of 6 layers, including a linear layer ( 98283 - > 